In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
def import_data(data_dir):
    train = pd.read_csv(data_dir + "/train.csv")
    X_train = train.values[:,1:]
    Y_train = train["label"]
    test = pd.read_csv(data_dir + "/test.csv")
    test = test.values[:]
    return X_train, Y_train, test

X_train, Y_train, X_test = import_data("../input")
print(X_train.shape, Y_train.shape, X_test.shape)

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)
Y_train = to_categorical(Y_train, num_classes = 10)
print(X_train.shape, Y_train.shape, X_test.shape)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

row = 9
#print(X_train[row].reshape(28,28))
g = plt.imshow(X_train[8][:,:,0])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_vali, y_train, y_vali = train_test_split(X_train, Y_train, test_size = 0.1, random_state = 0)
print(X_train.shape, X_vali.shape, y_train.shape, y_vali.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

epochs = 1 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 86

history = model.fit(x = X_train,y = y_train, batch_size=batch_size,
                              epochs = epochs, validation_data = (X_vali,y_vali),
                              verbose = 2)

In [ ]:
results = model.predict(X_test)

results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)
